# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 17/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***@homispeaker***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [2]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @homispeaker

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [6]:
#Produto escolhido:
produto = 'ifood'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [7]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [8]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [11]:
baseF = pd.read_excel('ifood_treinamento.xlsx')


i = 0
while i < len(baseF['Treinamento']):
    
    baseF['Treinamento'][i] = baseF['Treinamento'][i].replace('.',' ').replace('?',' ').replace(',',' ').replace(':',' ')\
.replace('(',' ').replace(')',' ').replace('!',' ').replace('"',' ').replace("'",' ').replace("®",' ').replace("/",' ')\
.replace("\n",' ').replace("+",' ').replace('...',' ').replace(';',' ')

    i+= 1
 
#----------------------------------------

i = 0    

while i < len(baseF['Treinamento']):
    slits = baseF['Treinamento'][i].split()
    a=0
    while a < len(slits):
        if(slits[a] == 'rt' or 'https' in slits[a] or '@' in slits[a]):
            #print('deletei',slits[a])
            del slits[a]
            
        a+=1
    baseF['Treinamento'][i] = ' '.join(slits)
    i+=1



palavras_all = []
relevancia = []
    

i=0 
while i < len(baseF):
    cache = baseF['Treinamento'][i].split()            
    for f in cache:
        palavras_all.append(f)
        relevancia.append(baseF['Relevância'][i])    
    
    i+=1
    
palavras = {'palavras': palavras_all,'relevancia': relevancia}
wordset = pd.DataFrame(data=palavras)


contagem = [['no',1,0]] #Lista com lista [palavra,vezes no alta,vezes no baixa]

i=1 
while i < len(wordset):
    for p in contagem:
        achou = False
        if wordset['palavras'][i] == p[0]:
            if wordset['relevancia'][i] == 'alta':
                p[1] += 1
                achou = True
                break
            else:
                p[2] += 1
                achou = True
                break
    if not achou:
        if(wordset['relevancia'][i] == 'alta'):
            contagem.append([wordset['palavras'][i],1,0])
        else:
            contagem.append([wordset['palavras'][i],0,1])
            
       
    i+=1

i = 0    
while i < len(contagem):
    
    if ('@' in contagem[i][0]):
        del contagem[i]
    
    i+= 1


In [20]:
twRelev = 0
twNRelev = 0 
twTotais = len(baseF)
for i in baseF['Relevância']:
    if(i == 'alta'):
        twRelev += 1
    else:
        twNRelev+=1
        
print('De um total de {0} tweets, {1} são relevantes e {2} são irrelevantes.'.format(twTotais,twRelev,twNRelev))
P_relev = twRelev/twTotais
P_irrelev = twNRelev/twTotais
print('Logo, a probabilidade de um tweet ser relevante é de {0}%, e de não ser relevante é de {1}%'.format(P_relev*100,P_irrelev))

De um total de 300 tweets, 98 são relevantes e 202 são irrelevantes.
Logo, a probabilidade de um tweet ser relevante é de 32.666666666666664%, e de não ser relevante é de 0.6733333333333333%


In [21]:
r = 0
nr = 0 
for k  in contagem:
    if k[1] > 0:
        r+=1
    if k[2] > 0:
        nr+=1
print('A quantidade de palavras relevantes é de {}, já a de irrelevantes é de {}'.format(r,nr))

A quantidade de palavras relevantes é de 499, já a de irrelevantes é de 935


In [27]:
#Começando o teorema em si:

def quant_usada(palavra,classify):
    for i in contagem:
        if palavra == i[0]:
            if(classify == 'alta'):
                return i[1]
            else:
                return i[2]
    return[0]
#Exemplo de uso: quant_usada('palavra a ser consultada', 'classificação desejada')

In [28]:
total_palavras = len(contagem)
def relevancia(frase):
    P_alta = 1
    P_baixa = 1    
    for t in frase.split():
        P_alta = P_alta*((quant_usada(t,'alta')+1)/r+total_palavras)
        P_baixa = P_baixa*((quant_usada(t,'baixa')+1)/nr+total_palavras)
    
    if P_alta > P_baixa:
        return 'alta'
    else:
        return 'baixa'

In [38]:
relevancia('ridículo esse comercial do ifood com a anitta 👊👊👊')

'alta'

,Treinamento,Relevância
0,no momento esperando um lindo xis strogonoff p...,alta
1,oi tudo bem quem define as opções de pagamento...,baixa
2,alguém compra alguma coisa pra eu comer no ifo...,alta
3,me patrocina me manda umas comida aqui em casa...,baixa
4,prisão domiciliar com essa grana do geddel dav...,baixa
5,ifood só me ferra tô com tédio aí entro no ifo...,alta
6,entrando no ifood e pesquisando comidas quem n...,alta
7,sacanagem t co xfk6pca24a,baixa
8,pedido feito no ifood agr é só ocupar a mente ...,alta
9,ridículo esse comercial do ifood com a anitta 👊👊👊,alta


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
